## Model

In [1]:
import torch
import torch.nn as nn
from transformers import SwinModel
from torchvision.models import convnext_base, ConvNeXt_Base_Weights
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from transformers import AutoImageProcessor
from PIL import Image
import numpy as np
import os
import pandas as pd
from sklearn.cluster import DBSCAN
import torch.nn.functional as F

2025-05-05 21:36:08.204973: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746480968.398443      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746480968.452211      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
class GeoLocationModel(nn.Module):
    def __init__(self, base_model_type='swin', region_num=15, embed_dim=64):
        super().__init__()
        self.region_embed = nn.Embedding(region_num, embed_dim)
        
        if base_model_type == 'swin':
            # Swin Transformer from Hugging Face
            self.image_encoder = SwinModel.from_pretrained(
                "microsoft/swin-base-patch4-window7-224"
            )
            img_feat_dim = self.image_encoder.config.hidden_size
        elif base_model_type == 'convnext':
            # ConvNeXt from TorchVision
            self.convnext = convnext_base(weights=ConvNeXt_Base_Weights.IMAGENET1K_V1)
            self.convnext.classifier = nn.Identity()
            # Create classifier processor to handle norm and pooling
            original_classifier = list(convnext_base().classifier.children())
            self.classifier_processor = nn.Sequential(*original_classifier[:-1])
            img_feat_dim = 1024  # For convnext-base
        else:
            raise ValueError("Invalid base_model_type. Choose 'swin' or 'convnext'")
        
        # Combined feature dimension
        combined_dim = img_feat_dim + embed_dim
        
        # Regression heads
        self.latitude_head = nn.Sequential(
            nn.Linear(combined_dim, 512),
            nn.ReLU(),
            nn.Linear(512, 1),
            nn.Sigmoid()
        )
        self.longitude_head = nn.Sequential(
            nn.Linear(combined_dim, 512),
            nn.ReLU(),
            nn.Linear(512, 1),
            nn.Sigmoid()
        )

    def forward(self, image, Region_ID):
        # Image feature extraction
        if hasattr(self, 'image_encoder'):  # Swin branch
            swin_output = self.image_encoder(image)
            img_features = swin_output.last_hidden_state.mean(dim=1)
        else:  # ConvNeXt branch
            conv_features = self.convnext(image)
            img_features = self.classifier_processor(conv_features)
        
        # Region embedding
        region_features = self.region_embed(Region_ID)
        
        # Combine features
        combined = torch.cat([img_features, region_features], dim=1)
        
        # Predict normalized coordinates
        lat = self.latitude_head(combined)
        lon = self.longitude_head(combined)
        return torch.cat([lat, lon], dim=1)

## Dataset

In [3]:
class GeoRegionDataset(Dataset):
    def __init__(self, dataframe, img_dir, transform=None, normalize_coords=True):
        """
        Args:
            dataframe: DataFrame with columns 'filename', 'latitude', 'longitude', 'Region_ID'
            img_dir: Path to image directory
            transform: Optional torchvision transforms
            normalize_coords: Whether to normalize coordinates to [0,1] range
        """
        self.img_paths = [os.path.join(img_dir, fname) for fname in dataframe['filename']]
        self.Region_IDs = dataframe['Region_ID'].values - 1
        
        # Convert coordinates to float32 and optionally normalize
        self.coordinates = dataframe[['latitude', 'longitude']].astype(np.float32).values
        self.original_coords = self.coordinates.copy()  # Store unnormalized coordinates
        if normalize_coords:
            self.coordinates = self.coordinates / 999999.0  # Normalize 6-digit integers to [0,1]
        
        self.transform = transform
        self.normalize_coords = normalize_coords

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        # Load image
        image = Image.open(self.img_paths[idx]).convert("RGB")
        
        # Apply transforms
        if self.transform:
            image = self.transform(image)
        
        # Get region ID (convert to tensor)
        Region_ID = torch.tensor(self.Region_IDs[idx], dtype=torch.long)
        
        # Get coordinates (already normalized if specified)
        coords = torch.tensor(self.coordinates[idx], dtype=torch.float32)
        original_coords = torch.tensor(self.original_coords[idx], dtype=torch.float32)
        
        return {
            'image': image,
            'Region_ID': Region_ID,
            'coordinates': coords,
            'original_coords': original_coords  # For denormalized metrics
        }

    @staticmethod
    def denoise_dataframe(df, coord_cols=['longitude', 'latitude'], mad_thresh=3.0, dbscan_eps=50, dbscan_min_samples=10):
        """
        Helper method to clean the input dataframe using MAD and DBSCAN
        Returns cleaned dataframe
        """
        # MAD filtering
        cleaned_df = df.copy()
        for col in coord_cols:
            med = cleaned_df[col].median()
            mad = np.median(np.abs(cleaned_df[col] - med))
            if mad > 0:  # Only filter if MAD is non-zero
                cleaned_df = cleaned_df[np.abs(cleaned_df[col] - med) <= mad_thresh * mad]
        
        # DBSCAN clustering
        if len(coord_cols) >= 2:
            coords = cleaned_df[coord_cols].values
            db = DBSCAN(eps=dbscan_eps, min_samples=dbscan_min_samples, metric='euclidean').fit(coords)
            cleaned_df['dbscan_label'] = db.labels_
            cleaned_df = cleaned_df[cleaned_df['dbscan_label'] != -1]
        
        return cleaned_df.drop(columns=['dbscan_label'], errors='ignore')


In [4]:
class SharedAugmentations:
    """Shared augmentation core for both models"""
    def __init__(self):
        self.transform = transforms.Compose([
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.RandomRotation(degrees=15),
            transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
        ])
    
    def __call__(self, img):
        return self.transform(img)

def create_model_specific_datasets(base_dataset, model_type):
    """Create model-specific datasets with proper transforms"""
    shared_aug = SharedAugmentations()
    
    if model_type == 'swin':
        processor = AutoImageProcessor.from_pretrained("microsoft/swin-base-patch4-window7-224")
        class SwinDataset(Dataset):
            def __init__(self, base_dataset):
                self.base_dataset = base_dataset
                self.processor = processor
            
            def __len__(self):
                return len(self.base_dataset)
            
            def __getitem__(self, idx):
                item = self.base_dataset[idx]
                img = shared_aug(item['image'])  # Apply shared augmentations
                return {
                    'image': self.processor(img, return_tensors="pt")['pixel_values'][0],
                    'Region_ID': item['Region_ID'],
                    'coordinates': item['coordinates'],
                    'original_coords': item['original_coords']
                }
        return SwinDataset(base_dataset)
    
    elif model_type == 'convnext':
        convnext_transform = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        class ConvNeXtDataset(Dataset):
            def __init__(self, base_dataset):
                self.base_dataset = base_dataset
                self.transform = convnext_transform
            
            def __len__(self):
                return len(self.base_dataset)
            
            def __getitem__(self, idx):
                item = self.base_dataset[idx]
                img = shared_aug(item['image'])  # Apply shared augmentations
                return {
                    'image': self.transform(img),
                    'Region_ID': item['Region_ID'],
                    'coordinates': item['coordinates'],
                    'original_coords': item['original_coords']
                }
        return ConvNeXtDataset(base_dataset)

### val prep

In [5]:
# Training and Validation Setup
def denormalize(coords_tensor):
    """Convert normalized [0,1] coords back to original 6-digit integers"""
    return coords_tensor * 999999.0

def validate(models, val_loaders, device):
    models['swin'].eval()
    models['convnext'].eval()
    
    # Initialize accumulators
    swin_sum_se = 0.0
    convnext_sum_se = 0.0
    count = 0
    
    with torch.no_grad():
        for swin_batch, convnext_batch in zip(val_loaders['swin'], val_loaders['convnext']):
            # Prepare data
            swin_images = swin_batch['image'].to(device)
            convnext_images = convnext_batch['image'].to(device)
            Region_IDs = swin_batch['Region_ID'].to(device)
            true_coords = swin_batch['original_coords'].to(device)
            
            # Get predictions
            swin_pred = denormalize(models['swin'](swin_images, Region_IDs))
            convnext_pred = denormalize(models['convnext'](convnext_images, Region_IDs))

            # Accumulate squared errors
            batch_size = swin_images.size(0)
            swin_sum_se += F.mse_loss(swin_pred, true_coords, reduction='sum').item()
            convnext_sum_se += F.mse_loss(convnext_pred, true_coords, reduction='sum').item()
            count += batch_size
    
    # Calculate MSEs
    swin_mse = swin_sum_se / count
    convnext_mse = convnext_sum_se / count
    
    # Calculate weights (with epsilon to avoid division by zero)
    eps = 1e-8
    swin_weight = 1 / (swin_mse + eps)
    convnext_weight = 1 / (convnext_mse + eps)
    total_weight = swin_weight + convnext_weight
    swin_weight /= total_weight
    convnext_weight /= total_weight
    
    # Estimated ensemble MSE (weighted average of errors)
    ensemble_mse = swin_weight**2 * swin_mse + convnext_weight**2 * convnext_mse
    
    return {
        'swin': swin_mse,
        'convnext': convnext_mse,
        'ensemble': ensemble_mse,
        'weights': {'swin': swin_weight, 'convnext': convnext_weight}
    }

### train prep

In [6]:
from tqdm.notebook import tqdm

# Main training function
def train_models():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    print("here")
    # Load and clean data
    train_df = GeoRegionDataset.denoise_dataframe(pd.read_csv(train_csv))
    val_df_noisy = pd.read_csv(val_csv)
    filtered_df = val_df_noisy[
        (val_df_noisy['latitude'] > 200_000) & (val_df_noisy['latitude'] < 230_000) &
        (val_df_noisy['longitude'] > 140_000) & (val_df_noisy['longitude'] < 150_000)
    ]
    val_df = filtered_df
    
    # Create base datasets
    base_train = GeoRegionDataset(train_df, train_img_dir, normalize_coords=True)
    base_val = GeoRegionDataset(val_df, val_img_dir, normalize_coords=True)
    
    # Create model-specific datasets
    train_datasets = {
        'swin': create_model_specific_datasets(base_train, 'swin'),
        'convnext': create_model_specific_datasets(base_train, 'convnext')
    }
    val_datasets = {
        'swin': create_model_specific_datasets(base_val, 'swin'),
        'convnext': create_model_specific_datasets(base_val, 'convnext')
    }
    
    # Create dataloaders
    print("here")
    batch_size = 32
    train_loaders = {
        'swin': DataLoader(train_datasets['swin'], batch_size=batch_size, shuffle=True, num_workers=4),
        'convnext': DataLoader(train_datasets['convnext'], batch_size=batch_size, shuffle=True, num_workers=4)
    }
    val_loaders = {
        'swin': DataLoader(val_datasets['swin'], batch_size=batch_size, shuffle=False),
        'convnext': DataLoader(val_datasets['convnext'], batch_size=batch_size, shuffle=False)
    }
    
    # Initialize models
    models = {
        'swin': GeoLocationModel(base_model_type='swin').to(device),
        'convnext': GeoLocationModel(base_model_type='convnext').to(device)
    }
    optimizers = {
        'swin': torch.optim.AdamW(models['swin'].parameters(), 
                                    lr=1e-4,
                                    weight_decay=0.01),
        'convnext': torch.optim.AdamW(models['convnext'].parameters(), lr=1e-4)
    }
    torch.nn.utils.clip_grad_norm_(models['swin'].parameters(), 1.0)

    criterion = nn.MSELoss()
    
    print("here")
    # Training loop
    num_epochs = 50
    best_ensemble_mse = float('inf')
    for epoch in range(num_epochs):
        # Training phase
        models['swin'].train()
        models['convnext'].train()
        total_batches = len(train_loaders['swin'])
        for (swin_batch, convnext_batch) in tqdm(zip(train_loaders['swin'], train_loaders['convnext']), total=total_batches):
            # Prepare data
            swin_images = swin_batch['image'].to(device)
            convnext_images = convnext_batch['image'].to(device)
            Region_IDs = swin_batch['Region_ID'].to(device)
            target_coords = swin_batch['coordinates'].to(device)
            
            # Train Swin
            optimizers['swin'].zero_grad()
            swin_output = models['swin'](swin_images, Region_IDs)
            swin_loss = criterion(swin_output, target_coords)
            swin_loss.backward()
            optimizers['swin'].step()
            
            # Train ConvNeXt
            optimizers['convnext'].zero_grad()
            convnext_output = models['convnext'](convnext_images, Region_IDs)
            convnext_loss = criterion(convnext_output, target_coords)
            convnext_loss.backward()
            optimizers['convnext'].step()
        
        # Validation phase
        val_metrics = validate(models, val_loaders, device)
        print(f"Epoch {epoch+1}/{num_epochs}:")
        print(f"  Swin Val MSE: {val_metrics['swin']:.2f}")
        print(f"  ConvNeXt Val MSE: {val_metrics['convnext']:.2f}")
        print(f"  Ensemble Val MSE: {val_metrics['ensemble']:.2f}\n")

        if val_metrics['ensemble'] < best_ensemble_mse:
            best_ensemble_mse = val_metrics['ensemble']
            torch.save({
                'epoch': epoch,
                'swin_state_dict': models['swin'].state_dict(),
                'convnext_state_dict': models['convnext'].state_dict(),
                'swin_optimizer': optimizers['swin'].state_dict(),
                'convnext_optimizer': optimizers['convnext'].state_dict(),
                'best_ensemble_mse': best_ensemble_mse
            }, '/kaggle/working/best_ensemble_model.pth')

    
    return models

## Running and Evaluating

In [7]:
train_csv = "/kaggle/input/smai-25-sec-a-project-phase-2-lat-long-prediction/labels_train.csv"
val_csv = "/kaggle/input/smai-25-sec-a-project-phase-2-lat-long-prediction/labels_val.csv"
train_img_dir = "/kaggle/input/smai-25-sec-a-project-phase-2-lat-long-prediction/images_train/images_train"
val_img_dir = "/kaggle/input/smai-25-sec-a-project-phase-2-lat-long-prediction/images_val/images_val"

# Load and clean data (only train needs cleaning)
train_df = pd.read_csv(train_csv)
val_df = pd.read_csv(val_csv)

# Execute training (this will automatically handle validation each epoch)
trained_models = train_models()

here


preprocessor_config.json:   0%|          | 0.00/255 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


here


config.json:   0%|          | 0.00/71.8k [00:00<?, ?B/s]

...

  0%|          | 0/183 [00:00<?, ?it/s]

Epoch 28/50:
  Swin Val MSE: 26735146.74
  ConvNeXt Val MSE: 9746139.93
  Ensemble Val MSE: 7142414.78



  0%|          | 0/183 [00:00<?, ?it/s]

Epoch 29/50:
  Swin Val MSE: 27896645.47
  ConvNeXt Val MSE: 10302263.95
  Ensemble Val MSE: 7523738.49



  0%|          | 0/183 [00:00<?, ?it/s]

In [ ]:
from scipy.stats import zscore

# Compute z-scores for both latitude and longitude
z_scores = val_df_noisy[['latitude', 'longitude']].apply(zscore)

# Keep only rows where both latitude and longitude z-scores are within 3 standard deviations
filtered_df = val_df_noisy[
    (val_df_noisy['latitude'] > 200_000) & (val_df_noisy['latitude'] < 230_000) &
    (val_df_noisy['longitude'] > 140_000) & (val_df_noisy['longitude'] < 150_000)
]
print(len(filtered_df))
# Plot the filtered data
plt.scatter(filtered_df['latitude'], filtered_df['longitude'], s=4)
plt.show()